<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#模型" data-toc-modified-id="模型-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>模型</a></span></li><li><span><a href="#Results" data-toc-modified-id="Results-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Results</a></span><ul class="toc-item"><li><span><a href="#my-notebook-1" data-toc-modified-id="my-notebook-1-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>my notebook 1</a></span><ul class="toc-item"><li><span><a href="#External-HDD,-BUP-Slim-BK-+-USB3.0" data-toc-modified-id="External-HDD,-BUP-Slim-BK-+-USB3.0-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>External HDD, BUP Slim BK + USB3.0</a></span></li><li><span><a href="#Internal-SSD" data-toc-modified-id="Internal-SSD-2.1.2"><span class="toc-item-num">2.1.2&nbsp;&nbsp;</span>Internal SSD</a></span></li></ul></li><li><span><a href="#my-notebook-2" data-toc-modified-id="my-notebook-2-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>my notebook 2</a></span></li><li><span><a href="#my-desktop-2" data-toc-modified-id="my-desktop-2-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>my desktop 2</a></span><ul class="toc-item"><li><span><a href="#External-HDD,-BUP-Slim-BK-+-USB3.0" data-toc-modified-id="External-HDD,-BUP-Slim-BK-+-USB3.0-2.3.1"><span class="toc-item-num">2.3.1&nbsp;&nbsp;</span>External HDD, BUP Slim BK + USB3.0</a></span></li></ul></li></ul></li></ul></div>

# 模型

我们使用torch官方的benchmark例子。

In [1]:
# % https://raw.githubusercontent.com/pytorch/examples/master/mnist/main.py
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, 1, padding=1)
        self.conv2 = nn.Conv2d(32, 128, 3, 1, padding=1)
        self.conv3 = nn.Conv2d(128, 512, 3, 1, padding=1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.dropout3 = nn.Dropout(0.5)
        
        self.fc1 = nn.Linear(32*32//4*512, 1024)
        self.fc2 = nn.Linear(1024, 128)
        self.fc3 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.conv3(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        x = self.dropout3(x)
        x = self.fc3(x)
        output = F.log_softmax(x, dim=1)
        return output


def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    
    
#     import torch.autograd.profiler as profiler
#     with profiler.profile(record_shapes=True) as prof:
#         with profiler.record_function("model_inference"):

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
#             print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
#                     epoch, batch_idx * len(data), len(train_loader.dataset),
#                     100. * batch_idx / len(train_loader), loss.item()))
            if args.dry_run:
                break

#     print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=10))

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0    
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    
    #print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


def main():
    # Training settings
    parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
    parser.add_argument('--batch-size', type=int, default=64, metavar='N',
                        help='input batch size for training (default: 64)')
    parser.add_argument('--test-batch-size', type=int, default=1000, metavar='N',
                        help='input batch size for testing (default: 1000)')
    parser.add_argument('--epochs', type=int, default=14, metavar='N',
                        help='number of epochs to train (default: 14)')
    parser.add_argument('--lr', type=float, default=1.0, metavar='LR',
                        help='learning rate (default: 1.0)')
    parser.add_argument('--gamma', type=float, default=0.7, metavar='M',
                        help='Learning rate step gamma (default: 0.7)')
    parser.add_argument('--no-cuda', action='store_true', default=False,
                        help='disables CUDA training')
    parser.add_argument('--dry-run', action='store_true', default=False,
                        help='quickly check a single pass')
    parser.add_argument('--seed', type=int, default=1, metavar='S',
                        help='random seed (default: 1)')
    parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                        help='how many batches to wait before logging training status')
    parser.add_argument('--save-model', action='store_true', default=False,
                        help='For Saving the current Model')
    args = parser.parse_args()
    use_cuda = not args.no_cuda and torch.cuda.is_available()

    torch.manual_seed(args.seed)

    device = torch.device("cuda" if use_cuda else "cpu")

    train_kwargs = {'batch_size': args.batch_size}
    test_kwargs = {'batch_size': args.test_batch_size}
    if use_cuda:
        cuda_kwargs = {'num_workers': 1,
                       'pin_memory': True,
                       'shuffle': True}
        train_kwargs.update(cuda_kwargs)
        test_kwargs.update(cuda_kwargs)

    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])
    dataset1 = datasets.CIFAR10 ('../data', train=True, download=True,
                       transform=transform)
    dataset2 = datasets.CIFAR10 ('../data', train=False,
                       transform=transform)
    train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs)
    test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

    model = Net().to(device)
    
    optimizer = optim.Adadelta(model.parameters(), lr=args.lr)

    scheduler = StepLR(optimizer, step_size=1, gamma=args.gamma)
    for epoch in range(1, args.epochs + 1):
        train(args, model, device, train_loader, optimizer, epoch)
        test(model, device, test_loader)
        scheduler.step()

    if args.save_model:
        torch.save(model.state_dict(), "mnist_cnn.pt")



In [2]:
import platform
import sys
import psutil
from torchvision import datasets


def list_env():
    print(platform.node())
    print(platform.platform())
    print(platform.machine())
    try:
        import cpuinfo
        print(cpuinfo.get_cpu_info()["brand_raw"])
        print("cpu-count", cpuinfo.get_cpu_info()["count"])
    except:
        print(platform.processor())
        print("cpu-count", psutil.cpu_count())
        
    print(sys.version)
    print(torch.__version__)
    print(torch.__config__.show())
    print(torch.__config__.parallel_info())

    
    print("gpu available:", torch.cuda.is_available())
    for i in range(torch.cuda.device_count()):
        print(torch.cuda.get_device_name(i))
        
        
def model_summary():            
    model = Net()
    for param in model.parameters():
        print(param.name, param.requires_grad,  param.size())

def download_data():
    dataset1 = datasets.MNIST('../data', train= True, download=True)
    dataset2 = datasets.MNIST('../data', train=False, download=True)
    
def benchmark(try_gpu=True, try_cpu=False):

    if try_gpu and torch.cuda.is_available():        
        print("device=gpu")
        psutil.cpu_percent(percpu=True)
        argv = sys.argv
        sys.argv = ["", "--epochs", "1", "--test-batch-size", "64"]
        %timeit -r1 -n1 main()
        sys.argv = argv
        print("cpu-percent:", psutil.cpu_percent(percpu=True))
        print()
        
    if try_cpu:
        print("device=cpu")
        psutil.cpu_percent(percpu=True)
        argv = sys.argv
        sys.argv = ["", "--epochs","1", "--no-cuda", "--test-batch-size", "64"]
        %timeit -r1 -n1 main()
        sys.argv = argv
        print("cpu-percent:", psutil.cpu_percent(percpu=True))
        print()
    
model_summary()
download_data()

None True torch.Size([32, 3, 3, 3])
None True torch.Size([32])
None True torch.Size([128, 32, 3, 3])
None True torch.Size([128])
None True torch.Size([512, 128, 3, 3])
None True torch.Size([512])
None True torch.Size([1024, 131072])
None True torch.Size([1024])
None True torch.Size([128, 1024])
None True torch.Size([128])
None True torch.Size([10, 128])
None True torch.Size([10])


# Results

## my notebook 1

In [4]:
list_env()

DESKTOP-U61NOQ2
Windows-10-10.0.18362-SP0
AMD64
Intel64 Family 6 Model 142 Stepping 10, GenuineIntel
cpu-count 8
3.8.5 (default, Sep  3 2020, 21:29:08) [MSC v.1916 64 bit (AMD64)]
1.7.1
PyTorch built with:
  - C++ Version: 199711
  - MSVC 192729112
  - Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191125 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v1.6.0 (Git Hash 5ef631a030a6f73131c77892041042805a06064f)
  - OpenMP 2019
  - CPU capability usage: AVX2
  - CUDA Runtime 10.2
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_61,code=sm_61;-gencode;arch=compute_70,code=sm_70;-gencode;arch=compute_75,code=sm_75;-gencode;arch=compute_37,code=compute_37
  - CuDNN 7.6.5
  - Magma 2.5.4
  - Build settings: BLAS=MKL, BUILD_TYPE=Release, CXX_FLAGS=/DWIN32 /D_WINDOWS /GR /EHsc /w /bigobj -openmp:experimental -DNDEBUG -DUSE_FBGEMM -DUSE_VULKAN_WRAPPER, PERF

### External HDD, BUP Slim BK + USB3.0

In [5]:
benchmark()
benchmark()

device=gpu
Test set: Average loss: 0.0438, Accuracy: 9856/10000 (99%)
22.7 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
cpu-percent: [34.9, 10.7, 40.5, 15.2, 32.1, 10.9, 18.7, 74.4]

device=cpu
Test set: Average loss: 0.0460, Accuracy: 9853/10000 (99%)
2min 12s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
cpu-percent: [46.0, 57.1, 60.2, 65.3, 46.0, 52.5, 60.1, 72.1]

device=gpu
Test set: Average loss: 0.0413, Accuracy: 9869/10000 (99%)
20.1 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
cpu-percent: [31.2, 9.9, 35.1, 10.6, 31.6, 8.9, 17.2, 77.7]

device=cpu
Test set: Average loss: 0.0460, Accuracy: 9853/10000 (99%)
2min 15s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
cpu-percent: [50.2, 60.3, 62.5, 69.5, 50.1, 52.9, 61.1, 74.2]



### Internal SSD

In [4]:
benchmark()
benchmark()

device=gpu
Test set: Average loss: 0.0440, Accuracy: 9858/10000 (99%)
22.5 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
cpu-percent: [22.4, 4.5, 46.0, 5.9, 19.5, 3.6, 8.6, 82.5]

device=cpu
Test set: Average loss: 0.0460, Accuracy: 9853/10000 (99%)
2min 16s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
cpu-percent: [53.1, 61.2, 66.5, 71.0, 48.1, 51.9, 59.5, 72.9]

device=gpu
Test set: Average loss: 0.0450, Accuracy: 9853/10000 (99%)
20.2 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
cpu-percent: [32.6, 10.4, 39.7, 13.9, 31.1, 9.5, 17.1, 78.5]

device=cpu
Test set: Average loss: 0.0460, Accuracy: 9853/10000 (99%)
2min 15s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
cpu-percent: [48.2, 59.2, 61.1, 73.1, 50.4, 52.0, 60.6, 73.6]



## my notebook 2

In [5]:
benchmark()

DESKTOP-JPH34GE
Windows-10-10.0.17134-SP0
Intel64 Family 6 Model 60 Stepping 3, GenuineIntel
cpu-count 8
3.8.5 (default, Sep  3 2020, 21:29:08) [MSC v.1916 64 bit (AMD64)]
1.7.1
gpu available: True
GeForce GTX 850M

device=gpu
Test set: Average loss: 0.0441, Accuracy: 9860/10000 (99%)
31.8 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
cpu-percent: [33.8, 14.1, 23.1, 8.2, 12.1, 9.7, 5.4, 61.7]

device=cpu
Test set: Average loss: 0.0460, Accuracy: 9853/10000 (99%)
2min 44s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
cpu-percent: [65.4, 45.8, 61.2, 59.0, 53.5, 59.0, 55.6, 69.9]


## my desktop 2

In [3]:
list_env()

DESKTOP-0J5HGI3
Windows-10-10.0.19041-SP0
AMD64
AMD64 Family 23 Model 113 Stepping 0, AuthenticAMD
cpu-count 16
3.8.5 (default, Sep  3 2020, 21:29:08) [MSC v.1916 64 bit (AMD64)]
1.7.1
PyTorch built with:
  - C++ Version: 199711
  - MSVC 192729112
  - Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191125 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v1.6.0 (Git Hash 5ef631a030a6f73131c77892041042805a06064f)
  - OpenMP 2019
  - CPU capability usage: AVX2
  - CUDA Runtime 11.0
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_61,code=sm_61;-gencode;arch=compute_70,code=sm_70;-gencode;arch=compute_75,code=sm_75;-gencode;arch=compute_80,code=sm_80;-gencode;arch=compute_37,code=compute_37
  - CuDNN 8.0.4
  - Magma 2.5.4
  - Build settings: BLAS=MKL, BUILD_TYPE=Release, CXX_FLAGS=/DWIN32 /D_WINDOWS /GR /EHsc /w /bigobj -openmp:experimental -DNDEBUG -DUS

### External HDD, BUP Slim BK + USB3.0

In [4]:
benchmark()
benchmark()

device=gpu
Files already downloaded and verified
Test set: Average loss: 1.2363, Accuracy: 5662/10000 (57%)
1min 5s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
cpu-percent: [6.1, 2.5, 7.0, 4.1, 3.0, 8.5, 6.9, 9.5, 3.7, 2.7, 4.5, 2.5, 3.5, 3.9, 3.6, 89.4]

device=gpu
Files already downloaded and verified
Test set: Average loss: 1.2316, Accuracy: 5727/10000 (57%)
1min 2s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
cpu-percent: [2.9, 0.7, 1.8, 6.5, 3.1, 1.7, 2.8, 3.2, 0.8, 0.8, 2.4, 1.0, 0.5, 1.9, 0.7, 95.7]

